In [2]:
import numpy as np
import random
lab_conc = np.load("lab_conc.npy", allow_pickle=True)
fea_conc = np.load("fea_conc.npy", allow_pickle=True)
mapping = np.load("mapping.npy")
end_index_lab = list(np.load("end_index_lab.npy"))

FileNotFoundError: [Errno 2] No such file or directory: 'end_index_lab.npy'

In [5]:
fea_conc.shape

(3511,)

In [6]:
lab_conc.shape

(3511,)

In [13]:
lab_filtered = lab_conc.copy()

for idx, sent in enumerate(fea_conc):
    fea_length = len(sent)
    lab_length = len(lab_conc[idx])
    if fea_length < lab_length:
        lab_filtered[idx] = lab_conc[idx][0:fea_length]
        #print("Length mismatch: feat = {}, lab = {}".format(fea_length, lab_length))

In [16]:
fea_cat = np.concatenate(fea_conc)

In [11]:
lab_cat = np.concatenate(lab_conc)

In [14]:
lab_cat = np.concatenate(lab_filtered)

In [15]:
lab_cat.shape

(337692,)

In [17]:
fea_cat.shape

(337692, 69)

In [18]:
lab_conc_lst = np.asarray(lab_conc)

In [19]:
lab_conc_lst.shape

(3511,)

In [2]:
(vals, counts) = np.unique(lab_conc, return_counts=True)
# Get features of zero-vectors
zero_feats = np.where(~mapping.any(axis=1))[0]
# Filter only the ones which actually occur in the labels
valid_zero_feats = [x for x in vals if x in zero_feats]
# Get the index of these in the values list (since some values may be missing)
zero_idx = np.nonzero(np.in1d(vals,valid_zero_feats))[0]

# Get the target number of counts for these zero-features as the
# mean of all the other counts
count_nonzero = np.delete(counts, zero_idx)
target_counts = int(count_nonzero.mean())
# Get the total number of zero-features
count_zero_total = counts[zero_idx].sum()

# Only remove if we have too many zero features
if count_zero_total > target_counts:
    # Step size for keeping elements
    step = int(count_zero_total/target_counts)
    # Get all indices with zero features.
    zero_indices = [np.where(lab_conc == zero)[0] for zero in valid_zero_feats]
    # Combine all zero indices into one
    zero_indices = np.concatenate(zero_indices)
    # This is all the zero indices except every step-size one
    idx_to_remove = np.delete(zero_indices, np.arange(0, zero_indices.size, step))
    # Remove these from the original labels
    lab_conc_filtered = np.delete(lab_conc, idx_to_remove)
else:
    lab_conc_filtered = lab_conc
np.unique(lab_conc_filtered, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35], dtype=int32),
 array([ 2666,  1397,   114,  3682,     9,   113,  3548,  1153,    21,
        17656,  1126,   995,   710,   726,  1184,   496,  1099,  2038,
        11134,  1786,  1122,   479,  6360,  2408,  4324,  3860,  1127,
         2621,  3653,   870,  5624,   648,  3664,  2110,   505]))

In [16]:
end_index_lab = list(np.load("end_index_lab.npy"))
print(end_index_lab[0:10])
print(end_index_lab[-10:])
print(idx_to_remove[0:100])
print(idx_to_remove[-100:])
def fix_end_labs(idx_to_remove, end_index_lab):
    # Where to start (updated each time to allow only one pass)
    start_idx = 0
    for idx, curr_end_lab in enumerate(end_index_lab):
        for removed_idx in idx_to_remove[start_idx:]:
            if removed_idx <= curr_end_lab:
                start_idx += 1
            else:
                end_index_lab[idx] -= start_idx
                break
    end_index_lab[-1] -= start_idx
    return end_index_lab
        
end_index_lab = fix_end_labs(idx_to_remove, end_index_lab)
print(end_index_lab[0:10])
print(end_index_lab[-10:])

[31, 62, 93, 124, 155, 186, 217, 248, 279, 310]
[140538, 140663, 140788, 140913, 141038, 141163, 141288, 141413, 141538, 141663]
[ 26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43
  44  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 126 127 128 129 130]
[141557 141558 141559 141560 141561 141563 141564 141565 141566 141567
 141568 141569 141570 141571 141572 141573 141574 141575 141576 141577
 141578 141579 141580 141581 141583 141584 141585 141586 141587 141588
 141589 141590 141591 141592 141593 141594 141595 141596 141597 141598
 141599 141600 141601 141603 141604 141605 141606 141607 141608 141609
 141610 141611 141612 141613 141614 141615 141616 141617 141618 141619
 141620 141621 141623 141624 141625 14

In [17]:
len(lab_conc_filtered)

91028